In [16]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch, BayesianOptimization
import tensorflow as tf

In [17]:
df_train = pd.read_csv('/home/ubuntu/cs230-project/data_sub_PCA/df_train_PCA_dict_df_metab.csv')
print (df_train.head)

<bound method NDFrame.head of      Depressed at Time  df_metab_0  df_metab_1  df_metab_2  df_metab_3  \
0                 True   45.789637   14.994401    2.792549   -7.301174   
1                False   21.305736  -12.739211    2.414824  -22.494078   
2                False  -11.160583    0.500590    1.948961  -18.053438   
3                False   -3.505071    3.023377  -14.520562    9.177950   
4                False    2.370956   14.234072    6.691360    4.755687   
..                 ...         ...         ...         ...         ...   
123              False    0.125152   -2.485553   -3.342023  -20.668625   
124              False  -16.114327  -22.152604  -23.709256  -18.545960   
125              False   36.381486    6.960738   11.266882    9.134153   
126              False   -2.210158   12.276662  -22.779149   -9.248137   
127              False   -1.710068   17.256411   11.794209    3.967151   

     df_metab_4  df_metab_5  df_metab_6  df_metab_7  df_metab_8  ...  \
0      2.

In [18]:
y_train = df_train.iloc[:,0]
X_train = df_train.iloc[:,1:]

In [19]:
# one-hot encoding
y_train = pd.get_dummies(y_train)[True]
print (y_train.sum())

11


In [20]:
print (y_train.shape)
print (X_train.shape)

(128,)
(128, 88)


In [21]:
df_dev = pd.read_csv('/home/ubuntu/cs230-project/data_sub_PCA/df_dev_PCA_dict_df_metab.csv')
print (df_dev.shape)

(37, 89)


In [22]:
y_dev = df_dev.iloc[:,0]
y_dev = pd.get_dummies(y_dev)[True]
X_dev = df_dev.iloc[:,1:]

In [23]:
print (y_dev.shape)
print (X_dev.shape)
print (y_dev.sum())

(37,)
(37, 88)
3


In [13]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units=40, activation='tanh',
                          kernel_regularizer = keras.regularizers.l2(hp.Float('l2_dns1',0, 0.02, step=0.001))))
    model.add(layers.Dropout(hp.Float('dropout1', 0, 0.5, step = 0.1)))
    model.add(layers.Dense(units=20, activation='relu',
                          kernel_regularizer = keras.regularizers.l2(hp.Float('l2_dns2',0, 0.02, step=0.001))))
    model.add(layers.Dropout(hp.Float('dropout2', 0, 0.5, step = 0.1)))
    model.add(layers.Dense(units=20, activation='relu',
                          kernel_regularizer = keras.regularizers.l2(hp.Float('l2_dns3',0, 0.02, step=0.001))))
    #model.add(layers.Dense(2, activation = 'softmax')) #2 for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))
    
    metrics_list = [
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
    ]
    
    model.compile(
        optimizer = keras.optimizers.RMSprop(
        learning_rate = 0.01,
        rho = 0.55,
        momentum = 0.5),
        loss = 'binary_crossentropy',
        metrics = metrics_list)
    
    return model

In [26]:
#tuner = RandomSearch(
tuner = BayesianOptimization(
    build_model,
    objective = 'val_binary_accuracy',
    max_trials = 5,
    executions_per_trial = 3,
    directory = 'Metabolomics_dev')

INFO:tensorflow:Reloading Oracle from existing project Metabolomics_dev/untitled_project/oracle.json


In [27]:
tuner.search(X_train.values, y_train.values,
            epochs=5,
            validation_data=(X_dev, y_dev))

Train on 128 samples, validate on 37 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 1.7017 - binary_accuracy: 0.6250 - precision: 0.1429 - recall: 1.00 - 0s 3ms/sample - loss: 1.4422 - binary_accuracy: 0.7891 - precision: 0.1000 - recall: 0.1818 - val_loss: 1.1379 - val_binary_accuracy: 0.9189 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 1.3172 - binary_accuracy: 0.8125 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 191us/sample - loss: 1.0644 - binary_accuracy: 0.9141 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.9363 - val_binary_accuracy: 0.9189 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 0.7963 - binary_accuracy: 0.9375 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 186us/sample - loss: 0.8490 - binary_accuracy: 0.9141 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.8112 - v

Train on 128 samples, validate on 37 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 1.0823 - binary_accuracy: 0.8438 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 3ms/sample - loss: 0.9343 - binary_accuracy: 0.8906 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.8296 - val_binary_accuracy: 0.9189 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 0.7412 - binary_accuracy: 0.9375 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 194us/sample - loss: 0.7547 - binary_accuracy: 0.9141 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.7245 - val_binary_accuracy: 0.9189 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 0.6440 - binary_accuracy: 0.9062 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 190us/sample - loss: 0.6882 - binary_accuracy: 0.9141 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_

Train on 128 samples, validate on 37 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 1.0522 - binary_accuracy: 0.9688 - precision: 1.0000 - recall: 0.50 - 0s 3ms/sample - loss: 1.0296 - binary_accuracy: 0.9219 - precision: 1.0000 - recall: 0.0909 - val_loss: 0.9419 - val_binary_accuracy: 0.9189 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 0.9365 - binary_accuracy: 0.8125 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 197us/sample - loss: 0.7631 - binary_accuracy: 0.9141 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.9081 - val_binary_accuracy: 0.9189 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 0.5921 - binary_accuracy: 0.9375 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 185us/sample - loss: 0.6136 - binary_accuracy: 0.9141 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.8020 - v

Train on 128 samples, validate on 37 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 0.7650 - binary_accuracy: 0.6562 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 3ms/sample - loss: 0.6096 - binary_accuracy: 0.8125 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.4755 - val_binary_accuracy: 0.9189 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 0.6187 - binary_accuracy: 0.8750 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 199us/sample - loss: 0.5000 - binary_accuracy: 0.9141 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.4757 - val_binary_accuracy: 0.9189 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 0.3320 - binary_accuracy: 0.9375 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 187us/sample - loss: 0.4202 - binary_accuracy: 0.9141 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_

Train on 128 samples, validate on 37 samples
Epoch 1/5
128/128 [==============================] - ETA: 0s - loss: 2.0492 - binary_accuracy: 0.2500 - precision: 0.0769 - recall: 1.00 - 0s 3ms/sample - loss: 1.6282 - binary_accuracy: 0.6562 - precision: 0.1333 - recall: 0.5455 - val_loss: 1.1551 - val_binary_accuracy: 0.9189 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
128/128 [==============================] - ETA: 0s - loss: 1.2657 - binary_accuracy: 0.8750 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 192us/sample - loss: 1.0972 - binary_accuracy: 0.9141 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.9431 - val_binary_accuracy: 0.9189 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/5
128/128 [==============================] - ETA: 0s - loss: 0.9434 - binary_accuracy: 0.9375 - precision: 0.0000e+00 - recall: 0.0000e+ - 0s 197us/sample - loss: 0.9144 - binary_accuracy: 0.9141 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.8047 - v

INFO:tensorflow:Oracle triggered exit


In [28]:
tuner.results_summary()

In [56]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print (best_hp.values)

{'dropout': 0.2, 'num_layers': 3, 'units_0': 40, 'act_0': 'tanh', 'learning_rate': 0.01, 'rho': 0.55, 'momentum': 0.5, 'units_1': 20, 'act_1': 'relu', 'units_2': 20, 'act_2': 'relu'}


In [30]:
best_model = tuner.get_best_models(num_models=1)[0]

In [57]:
print (best_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  3560      
_________________________________________________________________
dense_1 (Dense)              multiple                  820       
_________________________________________________________________
dense_2 (Dense)              multiple                  420       
_________________________________________________________________
dense_3 (Dense)              multiple                  42        
Total params: 4,842
Trainable params: 4,842
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
results = best_model.evaluate(X_dev.values, y_dev.values)

37/37 [==============================] - ETA: 0s - loss: 0.8509 - acc: 0.843 - 0s 186us/sample - loss: 0.7360 - acc: 0.8649
test loss, test acc: [0.7359883179998252, 0.8648649]


In [45]:
print ("test loss, test acc:", results)

test loss, test acc: [0.7359883179998252, 0.8648649]
